In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from datetime import datetime
import warnings

plt.rc('font', family='malgun gothic')
plt.rc('axes' , unicode_minus = False)
warnings.filterwarnings(action = 'ignore')

In [2]:
df = pd.read_parquet("C:/Users/whileduck/Desktop/Github/Concert-Hall-Price-Model/data/거래까지걸린시간_추가_클래식_데이터.parquet")
seat_information = pd.read_csv("C:/Users/whileduck/Desktop/Github/Concert-Hall-Price-Model/data/seat_information_angle_ver3.csv")
display(df.head())
print(df.shape)

,seat,X,Y,Z,층,블록,열,넘버,무대까지의 거리,좌우면적시야각,...,원등급추정,누적검색량,콘서트홀클래식_누적검색량,예술의전당클래식_누적검색량,서울클래식_누적검색량,좌석 오픈 여부,거래까지걸린시간,거래까지걸린시간_시간,거래까지걸린시간_등수,표준화 등수 점수
0,1층 A블록1열 1,1451,542,-93,1층,A블록,1,1,1551.713247,15.219900,...,2,1050.0,1000.0,0.0,50.0,1,19 days 15:06:00,471.1,38.0,0.339286
1,1층 A블록1열 2,1406,555,-93,1층,A블록,1,2,1514.433888,16.945594,...,2,1050.0,1000.0,0.0,50.0,1,19 days 15:06:00,471.1,38.0,0.339286
2,1층 A블록1열 3,1361,568,-93,1층,A블록,1,3,1477.698887,18.869853,...,2,1050.0,1000.0,0.0,50.0,1,19 days 15:06:00,471.1,38.0,0.339286
3,1층 A블록1열 4,1315,580,-93,1층,A블록,1,4,1440.234009,21.009736,...,2,1050.0,1000.0,0.0,50.0,1,19 days 15:06:00,471.1,38.0,0.339286
4,1층 A블록1열 5,1270,591,-93,1층,A블록,1,5,1403.862529,23.314833,...,2,1050.0,1000.0,0.0,50.0,1,19 days 15:06:00,471.1,38.0,0.339286


(378255, 54)


# **데이터 테이블 생성 (공연 별, 좌석 별)**

In [3]:
reservation_grouping = df.groupby('전체공연시간').agg({'좌석 오픈 여부' : np.sum, 
                                    '예매여부' : np.sum, '원가격추정':[np.min , np.mean , np.max],'거래까지걸린시간_시간' : [np.min,np.mean,np.max]}).reset_index()
reservation_grouping['공연 예매율'] = reservation_grouping['예매여부'] / reservation_grouping['좌석 오픈 여부'] # 공연 별 예매율 계산 

reservation_grouping = reservation_grouping.rename(columns = {'price': '공연 별 평균 가격'})

reservation_grouping.columns = [' '.join(col).strip() for col in reservation_grouping.columns.values]

display(reservation_grouping.sample(5))
print(reservation_grouping.shape)

,전체공연시간,좌석 오픈 여부 sum,예매여부 sum,원가격추정 amin,원가격추정 mean,원가격추정 amax,거래까지걸린시간_시간 amin,거래까지걸린시간_시간 mean,거래까지걸린시간_시간 amax,공연 예매율
123,2022-11-19 17:00:00,2505,982,30000,72039.920160,110000,15.016667,1770.802919,2105.366667,0.392016
105,2022-08-24 19:30:00,2505,764,20000,55261.477046,120000,153.016667,672.636409,788.366667,0.304990
54,2020-09-12 17:00:00,2322,167,55000,108796.407186,130000,14.000000,17.231637,38.100000,0.071921
140,2023-03-07 19:30:00,2505,2120,30000,69980.039920,120000,153.016667,919.856769,1171.150000,0.846307
4,2019-02-16 17:00:00,2505,1192,5000,26950.099800,50000,10.666667,864.852699,1073.016667,0.475848


(151, 10)


In [4]:
# 공연별 예매율 테이블 생성

cols = ['play_date', 'play_st_time','pre_open_date','open_date', 'running_time', 
        'intermission','전체공연시간','콘서트홀클래식_누적검색량','예술의전당클래식_누적검색량', '서울클래식_누적검색량'] # 합병해주자 


pre_table = pd.merge(reservation_grouping,df[cols], on = '전체공연시간', how = 'left').drop_duplicates() # merge 후 중복을 제거 

reservation_table = pre_table.reset_index(drop = True) # index 초기화 

In [5]:
# 공연별 요약 테이블에서 EDA 할 때 용이하도록 날짜도 포함해서 만들도록 하자 

reservation_table['연도'] = pd.to_datetime(reservation_grouping['전체공연시간']).map(lambda x: x.year)
reservation_table['월'] = pd.to_datetime(reservation_grouping['전체공연시간']).map(lambda x: x.month)
reservation_table['연 월'] = pd.to_datetime(reservation_table['연도'].astype(str) + '-' + reservation_table['월'].astype(str)) 
reservation_table['일'] = pd.to_datetime(reservation_grouping['전체공연시간']).map(lambda x: x.day)
reservation_table['시간'] = pd.to_datetime(reservation_grouping['전체공연시간']).map(lambda x: x.hour)
reservation_table['요일'] =  pd.to_datetime(reservation_table['play_date']).map(lambda x: x.weekday())  # 월요일 0 , 화요일 1 .... 일요일 6 으로 맵핑되도록 변경

In [6]:
reservation_table['총_누적_검색량'] = reservation_table['콘서트홀클래식_누적검색량'] + reservation_table['예술의전당클래식_누적검색량'] + reservation_table['서울클래식_누적검색량']

In [7]:
display(reservation_table.sample(5))
print(reservation_table.shape)

,전체공연시간,좌석 오픈 여부 sum,예매여부 sum,원가격추정 amin,원가격추정 mean,원가격추정 amax,거래까지걸린시간_시간 amin,거래까지걸린시간_시간 mean,거래까지걸린시간_시간 amax,공연 예매율,...,콘서트홀클래식_누적검색량,예술의전당클래식_누적검색량,서울클래식_누적검색량,연도,월,연 월,일,시간,요일,총_누적_검색량
139,2023-02-10 19:30:00,2505,1523,10000,61241.516966,130000,13.116667,368.753830,715.466667,0.607984,...,640.0,0.0,40.0,2023,2,2023-02-01,10,19,4,680.0
80,2021-11-13 17:00:00,1903,1166,50000,86884.231537,150000,15.016667,1117.558548,1240.283333,0.612717,...,560.0,0.0,20.0,2021,11,2021-11-01,13,17,5,580.0
99,2022-07-10 17:00:00,2505,811,40000,86463.073852,120000,14.033333,202.961406,473.016667,0.323752,...,1510.0,20.0,50.0,2022,7,2022-07-01,10,17,6,1580.0
9,2019-05-08 20:00:00,2505,1250,5000,85590.818363,110000,15.016667,1741.590013,2371.900000,0.499002,...,630.0,20.0,0.0,2019,5,2019-05-01,8,20,2,650.0
64,2021-05-25 19:30:00,2049,805,20000,53243.512974,105000,61.466667,414.224679,499.483333,0.392875,...,670.0,0.0,30.0,2021,5,2021-05-01,25,19,1,700.0


(151, 26)


# **좌석 데이터 요약 테이블 생성**

In [8]:
seat_grouping = df.groupby('seat').agg({'예매여부': np.sum, '좌석 오픈 여부': np.sum, 
                                        '원가격추정':np.mean, '거래까지걸린시간_시간':np.mean,
                                        '거래까지걸린시간_등수' : np.mean, '거래까지걸린시간_등수':[np.max, np.min],
                                        '표준화 등수 점수': np.mean}).reset_index()

seat_grouping['좌석 예매율'] = seat_grouping['예매여부'] / seat_grouping['좌석 오픈 여부']
seat_grouping = seat_grouping.rename(columns = {'price': '좌석 별 평균 가격'})


seat_grouping.columns = [' '.join(col).strip() for col in seat_grouping.columns.values]
pre_table = seat_grouping.reset_index(drop = True)

pre_table.sample(5)

,seat,예매여부 sum,좌석 오픈 여부 sum,원가격추정 mean,거래까지걸린시간_시간 mean,거래까지걸린시간_등수 amax,거래까지걸린시간_등수 amin,표준화 등수 점수 mean,좌석 예매율
2381,합창석 G블록3열 27,30,141,38907.284768,896.943333,985.0,3.0,0.131212,0.212766
1381,2층 BOX2 7,56,128,61854.304636,714.267857,985.0,3.0,0.260145,0.437500
429,1층 B블록6열 2,91,139,105198.675497,923.208425,908.0,1.0,0.320476,0.654676
490,1층 C블록10열 9,85,136,111589.403974,697.251765,935.0,1.0,0.358053,0.625000
80,1층 A블록18열 1,67,144,74437.086093,977.091294,967.0,1.0,0.189076,0.465278


In [9]:
# merge 하기 전에 이름을 변경해주자

seat_information = seat_information.rename(columns = {'전체_좌석' : 'seat'})

seat_table = pd.merge(pre_table, seat_information, how = 'left', on = 'seat')

display(seat_table.sample(5))

,seat,예매여부 sum,좌석 오픈 여부 sum,원가격추정 mean,거래까지걸린시간_시간 mean,거래까지걸린시간_등수 amax,거래까지걸린시간_등수 amin,표준화 등수 점수 mean,좌석 예매율,Unnamed: 0,...,무대까지의 거리,좌우면적시야각,상하면적시야각,시야방해점수,좌석공간넓음,휠체어석,1/거리**3,무대정면과의각도,좌우시선각도,상하시선각도
1284,2층 A블록3열 7,84,147,66589.403974,795.920437,967.0,1.0,0.273560,0.571429,1644,...,2570.083462,27.283754,14.440119,98,0,0,5.890584e-11,0.788,8.589410,16.199294
2412,합창석 G블록4열 24,28,145,38774.834437,749.556548,942.0,2.0,0.121023,0.193103,1482,...,1393.565571,47.877515,28.422031,84,0,0,3.695028e-10,-0.982,10.926546,18.839163
70,1층 A블록17열 10,88,151,79900.662252,822.841288,935.0,1.0,0.278618,0.582781,891,...,2578.445462,28.317791,12.193119,100,0,0,5.833459e-11,0.834,19.867630,4.805380
437,1층 B블록7열 1,96,150,103245.033113,814.880382,908.0,3.0,0.290694,0.640000,317,...,1604.156164,43.202370,21.753848,100,0,0,2.422479e-10,0.762,32.576781,0.428609
1488,2층 B블록7열 14,87,148,72516.556291,755.128161,749.0,3.0,0.283920,0.587838,1960,...,3076.612585,28.283112,5.916411,99,0,0,3.433852e-11,0.990,3.579421,16.775902


# **추후 특정 기준에 따라 요약 테이블을 생성하기 위해 함수로 만들어두자**

In [10]:
class ConditionSummaryTable:
    
    def __init__(self,seat_information):
        
        self.seat_information = seat_information
        self.seat_information = self.seat_information.rename(columns = {'전체_좌석' : 'seat'})
        
    def reservation_condition_table(self ,cond_df):
        
        
        self.reservation_grouping = cond_df.groupby('전체공연시간').agg({'좌석 오픈 여부' : np.sum, 
                                    '예매여부' : np.sum, '원가격추정':[np.min, np.mean, np.max],'거래까지걸린시간_시간' : [np.min,np.mean,np.max]}).reset_index()
        
        
        self.reservation_grouping['공연 예매율'] = self.reservation_grouping['예매여부'] / self.reservation_grouping['좌석 오픈 여부'] # 공연 별 예매율 계산 

        self.reservation_grouping = self.reservation_grouping.rename(columns = {'price': '공연 별 평균 가격'})

        self.reservation_grouping.columns = [' '.join(col).strip() for col in self.reservation_grouping.columns.values]

        # 공연별 예매율 테이블 생성

        cols = ['play_date', 'play_st_time','pre_open_date','open_date', 'running_time', 
                'intermission','전체공연시간','콘서트홀클래식_누적검색량','예술의전당클래식_누적검색량', '서울클래식_누적검색량'] # 합병해주자 


        pre_table = pd.merge(reservation_grouping,cond_df[cols], on = '전체공연시간', how = 'left').drop_duplicates() # merge 후 중복을 제거 

        self.reservation_table = pre_table.reset_index(drop = True) # index 초기화 
        
        # 공연별 요약 테이블에서 EDA 할 때 용이하도록 날짜도 포함해서 만들도록 하자 

        self.reservation_table['연도'] = pd.to_datetime(self.reservation_table['전체공연시간']).map(lambda x: x.year)
        self.reservation_table['월'] = pd.to_datetime(self.reservation_table['전체공연시간']).map(lambda x: x.month)
        self.reservation_table['연 월'] = pd.to_datetime(self.reservation_table['연도'].astype(str) + '-' + reservation_table['월'].astype(str)) 
        self.reservation_table['일'] = pd.to_datetime(self.reservation_table['전체공연시간']).map(lambda x: x.day)
        self.reservation_table['시간'] = pd.to_datetime(self.reservation_table['전체공연시간']).map(lambda x: x.hour)
        self.reservation_table['요일'] =  pd.to_datetime(self.reservation_table['전체공연시간']).map(lambda x: x.weekday())  # 월요일 0 , 화요일 1 .... 일요일 6 으로 맵핑되도록 변경
        
        return self.reservation_table
    
    def seat_information_condition_table(self,cond_df):
        
        
        self.seat_grouping = cond_df.groupby('seat').agg({'예매여부': np.sum, '좌석 오픈 여부': np.sum, 
                                        '원가격추정':np.mean, '거래까지걸린시간_시간':np.mean,
                                        '거래까지걸린시간_등수' : np.mean, '거래까지걸린시간_등수':[np.max, np.min],
                                        '표준화 등수 점수': np.mean}).reset_index()


        self.seat_grouping['좌석 예매율'] = self.seat_grouping['예매여부'] / self.seat_grouping['좌석 오픈 여부']
        self.seat_grouping = self.seat_grouping.rename(columns = {'price': '좌석 별 평균 가격'})


        self.seat_grouping.columns = [' '.join(col).strip() for col in self.seat_grouping.columns.values]
        
        self.pre_table = self.seat_grouping.reset_index(drop = True)
        

        self.seat_table = pd.merge(pre_table, self.seat_information, how = 'left', on = 'seat')
        
        return self.seat_table

In [11]:
file_path = 'C:/Users/whileduck/Desktop/Github/Concert-Hall-Price-Model/data/'

reservation_table.to_parquet(file_path + 'reservation_table.parquet',index = False)
seat_table.to_parquet(file_path + 'seat_table.parquet', index = False)